In [33]:
from PIL import Image
import glob
import re
from random import *
from os import listdir
from pickle import dump
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
import numpy as np
from keras.models import Model
from keras.models import load_model
from numpy import argmax
import cv2
import matplotlib.pyplot as plt
from keras.layers import Bidirectional
import cv2
from keras import optimizers
import networkx as nx
import sys
from datetime import datetime
from tf.keras.layers import Layer, Dense, TimeDistributed, Concatenate, InputSpec, Wrapper, RNN
from Utility.kulc import ExternalAttentionRNNWrapper


ImportError: No module named 'tf'

# ***Load description and features from file***

In [4]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text


# load clean descriptions into memory
def load_clean_descriptions(filename):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		# create list
		if image_id not in descriptions:
			descriptions[image_id] = list()
		# wrap description in tokens
		desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
		# store
		descriptions[image_id].append(desc)
	return descriptions

In [5]:
def load_photo_features(filename):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	#features = {k: all_features[k] for k in dataset}
	return all_features

# ***Tokenizing Descriptions***

In [6]:
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc
 
# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 


# ***Creating sequence for LSTM***

In [7]:
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)
 
def create_sequences(tokenizer, max_length, descriptions, photos, vocab_size):
	X1, X2, y = list(), list(), list()
	# walk through each image identifier
	for key, desc_list in descriptions.items():
		# walk through each description for the image
		for desc in desc_list:
			# encode the sequence
			seq = tokenizer.texts_to_sequences([desc])[0]
			# split one sequence into multiple X,y pairs
			for i in range(1, len(seq)):
				# split into input and output pair
				in_seq, out_seq = seq[:i], seq[i]
				# pad input sequence
				in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
				# encode output sequence
				out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
        
				X1.append(photos[int(key)][0])
				X2.append(in_seq)
				y.append(out_seq)
	return np.array(X1), np.array(X2), np.array(y)


# ***Kulc KERAS EXTENSION WRAPPER***

# ***LSTM MODEL***

In [12]:
# define the captioning model
def define_model(vocab_size, max_length,embeddings_size):
	# feature extractor model
	inputs1 = Input(shape=(4096,))
	fe1 = Dropout(0.5)(inputs1)
	fe2 = Dense(256, activation='relu')(fe1)
	# sequence model
	inputs2 = Input(shape=(max_length,))
	se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
	se2 = Dropout(0.5)(se1)
	se3 = Bidirectional(LSTM(128))(se2)
	print(type(se3))# decoder model
	decoder1 = add([fe2, se3])
	decoder2 = Dense(256, activation='relu')(decoder1)
	outputs = Dense(vocab_size, activation='softmax')(decoder2)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	model.compile(loss='categorical_crossentropy', optimizer= optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False))
	# summarize model
	print(model.summary())
	plot_model(model, to_file='model.png', show_shapes=True)
	return model

In [ ]:
W = 14
H = 14
L = W*H
D = 512



def create_model(vocabulary_size, embedding_size, T, L, D):
    image_features_input = Input(shape=(L, D), name="image_features_input")
    captions_input = Input(shape=(T,), name="captions_input")
    captions = Embedding(vocabulary_size, embedding_size, input_length=T)(captions_input)

    averaged_image_features = Lambda(lambda x: K.mean(x, axis=1))
    averaged_image_features = averaged_image_features(image_features_input)
    initial_state_h = Dense(embedding_size)(averaged_image_features)
    initial_state_c = Dense(embedding_size)(averaged_image_features)

    image_features = TimeDistributed(Dense(D, activation="relu"))(image_features_input)

    encoder = LSTM(embedding_size, return_sequences=True, return_state=True, recurrent_dropout=0.1)
    attented_encoder = ExternalAttentionRNNWrapper(encoder, return_attention=True)

    output = TimeDistributed(Dense(vocabulary_size, activation="softmax"), name="output")

    # for training purpose
    attented_encoder_training_data, _, _ , _= attented_encoder([captions, image_features], initial_state=[initial_state_h, initial_state_c])
    training_output_data = output(attented_encoder_training_data)

    training_model = Model(inputs=[captions_input, image_features_input], outputs=training_output_data)
    
    initial_state_inference_model = Model(inputs=[image_features_input], outputs=[initial_state_h, initial_state_c])
    
    inference_initial_state_h = Input(shape=(embedding_size,))
    inference_initial_state_c = Input(shape=(embedding_size,))
    attented_encoder_inference_data, inference_encoder_state_h, inference_encoder_state_c, inference_attention = attented_encoder(
        [captions, image_features],
        initial_state=[inference_initial_state_h, inference_initial_state_c]
        )
   
    inference_output_data = output(attented_encoder_inference_data)

    inference_model = Model(
        inputs=[image_features_input, captions_input, inference_initial_state_h, inference_initial_state_c],
        outputs=[inference_output_data, inference_encoder_state_h, inference_encoder_state_c, inference_attention]
    )
    
    return training_model, inference_model, initial_state_inference_model


# ***Making train set***

In [8]:
train_descriptions = load_clean_descriptions('../Description_Text/youtube_video_img2_train.txt')
print('Descriptions: train=%d' % len(train_descriptions))
#photo features
train_features = load_photo_features('../Features_pkl/youtube_video_img2_train.pkl')
print('Photos: train=%d' % len(train_features))
# print(train_features[0])

Descriptions: train=17066
Photos: train=17066


In [9]:
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)


Vocabulary Size: 60
Description Length: 9


In [10]:
# prepare sequences
X1train, X2train, ytrain = create_sequences(tokenizer, max_length, train_descriptions, train_features, vocab_size)

# ***Making Validation Set***

In [17]:
train_descriptions_dev = load_clean_descriptions('../Description_Text/new_youtube_Img_val.txt')
print('Descriptions: train=%d' % len(train_descriptions_dev))
#photo feature

train_features_dev = load_photo_features('../Features_pkl/New_youtube_Img2_val.pkl')
print('Photos: train=%d' % len(train_features_dev))
# print(train_features[0][0])

Descriptions: train=35
Photos: train=35


In [18]:
X1test, X2test, ytest = create_sequences(tokenizer, max_length, train_descriptions_dev, train_features_dev, vocab_size)

# ***Training Model***

In [19]:


# define the model
model = define_model(vocab_size, max_length)
# define checkpoint callback
filepath = '../Models/model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}_VGG19_new_img2.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# fit model
model.fit([X1train, X2train], ytrain, epochs=1000, verbose=2, callbacks=[checkpoint], validation_data=([X1test, X2test], ytest))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
<class 'tensorflow.python.framework.ops.Tensor'>
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 9)            0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 9, 256)       15360       input_2[0][0]                    
__________________________________________________________________


Epoch 00033: val_loss did not improve from 0.10811
Epoch 34/1000
 - 148s - loss: 0.0965 - val_loss: 0.1083

Epoch 00034: val_loss did not improve from 0.10811
Epoch 35/1000
 - 148s - loss: 0.0965 - val_loss: 0.1093

Epoch 00035: val_loss did not improve from 0.10811
Epoch 36/1000
 - 148s - loss: 0.0964 - val_loss: 0.1084

Epoch 00036: val_loss did not improve from 0.10811
Epoch 37/1000
 - 148s - loss: 0.0962 - val_loss: 0.1084

Epoch 00037: val_loss did not improve from 0.10811
Epoch 38/1000
 - 147s - loss: 0.0964 - val_loss: 0.1082

Epoch 00038: val_loss did not improve from 0.10811
Epoch 39/1000
 - 152s - loss: 0.0964 - val_loss: 0.1083

Epoch 00039: val_loss did not improve from 0.10811
Epoch 40/1000
 - 151s - loss: 0.0962 - val_loss: 0.1084

Epoch 00040: val_loss did not improve from 0.10811
Epoch 41/1000
 - 150s - loss: 0.0964 - val_loss: 0.1086

Epoch 00041: val_loss did not improve from 0.10811
Epoch 42/1000
 - 149s - loss: 0.0962 - val_loss: 0.1086

Epoch 00042: val_loss did n

 - 149s - loss: 0.0960 - val_loss: 0.1083

Epoch 00107: val_loss did not improve from 0.10793
Epoch 108/1000
 - 149s - loss: 0.0959 - val_loss: 0.1080

Epoch 00108: val_loss did not improve from 0.10793
Epoch 109/1000
 - 150s - loss: 0.0960 - val_loss: 0.1079

Epoch 00109: val_loss did not improve from 0.10793
Epoch 110/1000
 - 148s - loss: 0.0959 - val_loss: 0.1080

Epoch 00110: val_loss did not improve from 0.10793
Epoch 111/1000
 - 149s - loss: 0.0960 - val_loss: 0.1080

Epoch 00111: val_loss did not improve from 0.10793
Epoch 112/1000
 - 149s - loss: 0.0960 - val_loss: 0.1082

Epoch 00112: val_loss did not improve from 0.10793
Epoch 113/1000
 - 149s - loss: 0.0961 - val_loss: 0.1079

Epoch 00113: val_loss did not improve from 0.10793
Epoch 114/1000
 - 148s - loss: 0.0959 - val_loss: 0.1081

Epoch 00114: val_loss did not improve from 0.10793
Epoch 115/1000
 - 149s - loss: 0.0960 - val_loss: 0.1082

Epoch 00115: val_loss did not improve from 0.10793
Epoch 116/1000
 - 149s - loss: 0.0

 - 150s - loss: 0.0960 - val_loss: 0.1079

Epoch 00180: val_loss did not improve from 0.10792
Epoch 181/1000
 - 150s - loss: 0.0959 - val_loss: 0.1079

Epoch 00181: val_loss did not improve from 0.10792
Epoch 182/1000
 - 150s - loss: 0.0959 - val_loss: 0.1081

Epoch 00182: val_loss did not improve from 0.10792
Epoch 183/1000
 - 149s - loss: 0.0960 - val_loss: 0.1080

Epoch 00183: val_loss did not improve from 0.10792
Epoch 184/1000
 - 150s - loss: 0.0958 - val_loss: 0.1079

Epoch 00184: val_loss did not improve from 0.10792
Epoch 185/1000
 - 149s - loss: 0.0960 - val_loss: 0.1080

Epoch 00185: val_loss did not improve from 0.10792
Epoch 186/1000
 - 149s - loss: 0.0960 - val_loss: 0.1080

Epoch 00186: val_loss did not improve from 0.10792
Epoch 187/1000
 - 149s - loss: 0.0958 - val_loss: 0.1079

Epoch 00187: val_loss did not improve from 0.10792
Epoch 188/1000
 - 150s - loss: 0.0960 - val_loss: 0.1080

Epoch 00188: val_loss did not improve from 0.10792
Epoch 189/1000
 - 149s - loss: 0.0

KeyboardInterrupt: 